# Face Swap with Eating/Licking Preservation
Preserves mouth actions like eating, licking, sucking candy

In [ ]:
# Setup
!apt update -qq && apt install -y ffmpeg
!git clone https://github.com/Mayank-kanojiya/liveswap.git
%cd liveswap
!pip install -q opencv-python onnxruntime-gpu torch torchvision insightface
!pip install -q customtkinter pillow psutil opennsfw2 protobuf cv2-enumerate-cameras
!pip install -q git+https://github.com/xinntao/BasicSR.git@master
!pip install -q git+https://github.com/TencentARC/GFPGAN.git@master

In [ ]:
# Download models
import os
os.makedirs('models', exist_ok=True)
!wget -q -O models/GFPGANv1.4.pth https://huggingface.co/hacksider/deep-live-cam/resolve/main/GFPGANv1.4.pth
!wget -q -O models/inswapper_128_fp16.onnx https://huggingface.co/hacksider/deep-live-cam/resolve/main/inswapper_128_fp16.onnx

In [ ]:
# Face swap with eating preservation
import cv2
import numpy as np
import sys
sys.path.append('.')

import modules.globals
modules.globals.headless = True
modules.globals.execution_providers = ['CUDAExecutionProvider', 'CPUExecutionProvider']
modules.globals.frame_processors = ['face_swapper']

from modules.processors.frame.face_swapper import get_face_swapper, process_frame
from modules.face_analyser import get_one_face
from modules.utilities import is_video, extract_frames, create_video, restore_audio, get_temp_frame_paths, create_temp, clean_temp

face_swapper = get_face_swapper()

def create_eating_mask(landmarks, image_shape):
    """Create extended mouth mask for eating/licking actions"""
    try:
        # Mouth region from 106-point landmarks (points 52-71)
        mouth_points = landmarks[52:72]
        
        # Create base mask
        mask = np.zeros(image_shape[:2], dtype=np.uint8)
        cv2.fillPoly(mask, [mouth_points.astype(np.int32)], 255)
        
        # Extend mask significantly for eating actions
        kernel = np.ones((25,25), np.uint8)
        mask = cv2.dilate(mask, kernel, iterations=8)
        
        # Add large circular area around mouth for objects
        mouth_center = np.mean(mouth_points, axis=0).astype(int)
        cv2.circle(mask, tuple(mouth_center), 60, 255, -1)
        
        # Smooth edges
        mask = cv2.GaussianBlur(mask, (31, 31), 0)
        
        return mask
    except:
        return None

def face_swap_preserve_eating(source_path, target_path, output_path, preserve_mouth=True):
    modules.globals.source_path = source_path
    modules.globals.target_path = target_path
    modules.globals.output_path = output_path
    
    source_img = cv2.imread(source_path)
    source_face = get_one_face(source_img)
    
    if is_video(target_path):
        print("Processing video - preserving eating actions...")
        create_temp(target_path)
        extract_frames(target_path)
        
        temp_frame_paths = get_temp_frame_paths(target_path)
        
        for i, frame_path in enumerate(temp_frame_paths):
            frame = cv2.imread(frame_path)
            target_face = get_one_face(frame)
            
            # Face swap
            result_frame = process_frame(source_face, frame)
            
            # Preserve eating/mouth actions
            if preserve_mouth and target_face and hasattr(target_face, 'landmark_2d_106'):
                eating_mask = create_eating_mask(target_face.landmark_2d_106, frame.shape)
                if eating_mask is not None:
                    # Convert mask to 3-channel
                    mask_3d = cv2.merge([eating_mask, eating_mask, eating_mask]) / 255.0
                    # Blend original mouth/eating area back
                    result_frame = result_frame * (1 - mask_3d) + frame * mask_3d
                    result_frame = result_frame.astype(np.uint8)
            
            cv2.imwrite(frame_path, result_frame)
            
            if i % 30 == 0:
                print(f"Processed {i+1}/{len(temp_frame_paths)} frames")
        
        create_video(target_path)
        restore_audio(target_path, output_path)
        clean_temp(target_path)
        
    else:
        print("Processing image - preserving eating actions...")
        target_img = cv2.imread(target_path)
        target_face = get_one_face(target_img)
        
        # Face swap
        result_frame = process_frame(source_face, target_img)
        
        # Preserve eating actions
        if preserve_mouth and target_face and hasattr(target_face, 'landmark_2d_106'):
            eating_mask = create_eating_mask(target_face.landmark_2d_106, target_img.shape)
            if eating_mask is not None:
                mask_3d = cv2.merge([eating_mask, eating_mask, eating_mask]) / 255.0
                result_frame = result_frame * (1 - mask_3d) + target_img * mask_3d
                result_frame = result_frame.astype(np.uint8)
        
        cv2.imwrite(output_path, result_frame)
    
    print(f"Face swap with eating preservation completed: {output_path}")

print("Face swap with eating/licking preservation ready!")

In [ ]:
# Usage - preserves eating, licking, sucking candy actions
source_image = "/content/source.jpg"
target_media = "/content/eating_video.mp4"  # Video of person eating/licking
output_file = "/content/result_with_eating.mp4"

# Face swap while preserving eating actions
face_swap_preserve_eating(source_image, target_media, output_file, preserve_mouth=True)

# Download result
from google.colab import files
files.download(output_file)